In [ ]:
import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False
rcParams["font.family"] = 'DejaVu Sans'
rcParams["font.size"] = 12

# Static

In [ ]:
from wall_analysis import parse_experiments

exp_name = '2021_09_23_polar_measurement'

fname = f'../experiments/{exp_name}/all_data.pkl'
try:
    all_df = pd.read_pickle(fname)
    print('read', fname)
except:
    all_df = parse_experiments(exp_name, verbose=False)
    all_df.to_pickle(fname)

In [ ]:
amplitude_method = "fixed" # window of 3 around source frequency
#amplitude_method = "maximum" 
mic_idx = 0 # facing speaker
#mic_idx = 1 # facing speaker
#mic_idx = 3 # away from speaker

for motors, df_motor in all_df.groupby('motors'):
    fig, ax = plt.subplots()
    for source, df in df_motor.groupby('source'):
        angles = []
        amplitudes = []
        
        freq = int(source.replace("mono", ""))

        for angle, df_angle in df.groupby('degree'):
            row = df_angle.iloc[0]
            
            angles.append(angle)

            stft = np.abs(row.stft) # times x n_mics x n_mics
            spectrogram = stft[:, mic_idx, :].T # freqs x times
            freqs = row.frequencies_matrix[0, :]

            energy = np.sum(spectrogram, axis=0)
            max_energy = np.max(energy)
            #start = np.where(energy > 0.1 * max_energy)[0][0]
            start = 3
            #cutoff = np.where(energy < 0.1 * max_energy)[0]
            cutoff = 23 #len(energy) - 1
            #cutoff = cutoff[cutoff > start][0]
            
            if amplitude_method == "maximum":
                amplitude = np.mean(np.max(spectrogram[:, start:cutoff], axis=0))
            elif amplitude_method == "fixed":
                freq_idx = np.argmin(np.abs(freqs - freq))
                amplitude = np.mean(spectrogram[freq_idx-1:freq_idx:2, start:cutoff].flatten())
            amplitudes.append(amplitude)
            
        ax.plot(angles, amplitudes, label=source)
    ax.set_xlabel('angle [deg]')
    ax.set_ylim(0.1, 10)
    ax.set_ylabel('average maximum amplitude')
    ax.legend(title='source', loc='upper left', bbox_to_anchor=[1, 1])
    ax.set_title(f'motors {motors}')
    ax.set_yscale('log')

In [ ]:
mic_idx = 0 # facing speaker
#mic_idx = 1 # facing speaker
#mic_idx = 3 # away from speaker

for motors, df_motor in all_df.groupby('motors'):
    for source, df in df_motor.groupby('source'):
        freq = int(source.replace("mono", ""))
        
        fig, ax_specs = plt.subplots(1, len(df.degree.unique()), sharey=True)
        fig.suptitle(f'source {source}, motors {motors}')
        fig.set_size_inches(15, 5)
        i_angle = 0
        for angle, df_angle in df.groupby('degree'):
            row = df_angle.iloc[0]

            stft = np.abs(row.stft) # times x n_mics x n_mics
            spectrogram = stft[:, mic_idx, :].T # freqs x times
            freqs = row.frequencies_matrix[0, :]
            invalid = np.where(row.frequencies_matrix[[0], :]!=freqs[None, :])[0]
            if len(invalid) > 0:
                print('invalid times:', invalid)

            energy = np.sum(spectrogram, axis=0)
            max_energy = np.max(energy)
            #start = np.where(energy > 0.1 * max_energy)[0][0]
            start = 3
            #cutoff = np.where(energy < 0.1 * max_energy)[0]
            cutoff = 23 #len(energy) - 1
            #cutoff = cutoff[cutoff > start][0]
            
            ax_specs[i_angle].pcolorfast(row.seconds, freqs, spectrogram[:-1, :-1])
            ax_specs[i_angle].set_title(f'{angle}')
            ax_specs[i_angle].axvline(row.seconds[start], color='white')
            ax_specs[i_angle].axvline(row.seconds[cutoff], color='white')
            i_angle += 1

# Moving

In [ ]:
from wall_analysis import parse_experiments

exp_name = '2021_09_30_polar_measurement'

fname = f'../experiments/{exp_name}/all_data.pkl'
try:
    raise
    all_df = pd.read_pickle(fname)
    print('read', fname)
except:
    all_df = parse_experiments(exp_name, verbose=False)
    all_df.to_pickle(fname)
all_df

In [ ]:
mic_idx = 3
vmax = 10
w = 5

chosen_df = all_df.loc[
    (all_df.appendix == '_loud') & 
    (all_df.source == 'mono3000') &
    (all_df.motors == 0)
]

for (motors, source, appendix), df in chosen_df.groupby(['motors', 'source', 'appendix']):
    assert len(df) == 1
    row = df.iloc[0]

    freq = int(row.source.strip("mono"))

    spectrogram = np.abs(row.stft[:, mic_idx, :].T)
    freqs = row.frequencies_matrix[-1, :]
    freq_idx = np.argmin(np.abs(freqs - freq))
    
    #amplitudes = spectrogram[freq_idx, :]
    amplitudes = np.max(spectrogram, axis=0)
    
    times = row.seconds
    angles = row.positions[:, 3] #np.cumsum(row.positions[:, 3]) * np.r_[0, times[1:] - times[:-1]] * 180 / np.pi

    #fig, ax = plt.subplots()
    #ax.plot(times, angles)
    fig, ax = plt.subplots()
    fig.set_size_inches(5, 3)
    ax.pcolorfast(times[:20], freqs, np.log10(spectrogram[:-1, :19]), vmax=np.log10(vmax))

    fig, axs = plt.subplots(1, 3)
    fig.set_size_inches(15, 3)
    axs[0].pcolorfast(times, freqs, np.log10(spectrogram[:-1, :-1]), vmax=np.log10(vmax))
    title = source
    title += ', no motors, ' if motors == 0 else ', motors, '
    title += appendix.strip("_")
    axs[0].set_title(title)
    axs[1].set_title(f'amplitude at {freqs[freq_idx]:.0f} Hz')
    axs[1].scatter(times, amplitudes)
    
    amplitudes_sum = np.mean(spectrogram[freq_idx-w:freq_idx+w, :], axis=0)
    axs[0].axhline(freqs[freq_idx-w], color='white')
    axs[0].axhline(freqs[freq_idx+w], color='white')
    axs[0].set_title(title)
    axs[2].set_title(f'mean amplitudes in {freqs[freq_idx-w]:.0f}:{freqs[freq_idx+w]:.0f} Hz')
    axs[2].scatter(times, amplitudes_sum)
    #axs[1].set_yscale('log')
    #axs[1].set_ylim(0, vmax)